In [ ]:

# prompt: kaggle datasets download -d ohagwucollinspatrick/ghana-crop-disease

!kaggle datasets download -d ohagwucollinspatrick/ghana-crop-disease

Dataset URL: https://www.kaggle.com/datasets/ohagwucollinspatrick/ghana-crop-disease
License(s): Attribution 4.0 International (CC BY 4.0)
ghana-crop-disease.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip ghana-crop-disease.zip -d data


Archive:  ghana-crop-disease.zip
replace data/SampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Load train and test files
train = pd.read_csv('/content/data/Train.csv')
test = pd.read_csv('/content/data/Test.csv')


# Afficher les premières lignes du fichier d'entraînement
print("Train Data:")
print(train.head())

# Afficher les premières lignes du fichier de test
print("\nTest Data:")
print(test.head())

# Afficher les informations sur les colonnes
print("\nTrain Data Info:")
print(train.info())

In [ ]:

# Charger les données
#train = pd.read_csv('/content/Train.csv')

# Visualiser la distribution des classes
plt.figure(figsize=(10,6))
sns.countplot(x='class', data=train)
plt.title('Distribution des classes dans le dataset de formation')
plt.xticks(rotation=90)
plt.show()

# Analyser les dimensions des images (si disponibles)
train['image_path'] = '/content/images/' + train['Image_ID']

In [ ]:
# Calculer la fréquence des classes
class_counts = train['class'].value_counts()
total_count = len(train)

# Calculer les poids en inversant la fréquence de chaque classe
class_weights = {cls: total_count / count for cls, count in class_counts.items()}

# Afficher les poids attribués à chaque classe
print("Class Weights:", class_weights)


In [ ]:
import pandas as pd
df = pd.read_csv('/content/data/Train.csv')
unique_classes = df['class'].unique()
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
print(class_mapping)


In [ ]:
# Import libraries
!pip install ultralytics
import pandas as pd
import os
from pathlib import Path
import shutil
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import cv2
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
import multiprocessing



# Set the data directory
DATA_DIR = Path('/content/data')

# Load train and test files
train = pd.read_csv(DATA_DIR / 'Train.csv')
test = pd.read_csv(DATA_DIR / 'Test.csv')
ss = pd.read_csv(DATA_DIR / 'SampleSubmission.csv')

# Add an image_path column
train['image_path'] = [Path('/content/data/images/' + x) for x in train.Image_ID]
test['image_path'] = [Path('/content/data/images/' + x) for x in test.Image_ID]

# Map string classes to integers (label encoding targets)
train['class_id'] = train['class'].map(
    {'Pepper_Bacterial_Spot': 0, 'Pepper_Fusarium': 1, 'Corn_Cercospora_Leaf_Spot': 2, 'Corn_Common_Rust': 3, 'Tomato_Early_Blight': 4, 'Pepper_Septoria': 5, 'Tomato_Septoria': 6, 'Pepper_Leaf_Curl': 7, 'Pepper_Leaf_Mosaic': 8, 'Corn_Streak': 9, 'Corn_Healthy': 10, 'Pepper_Healthy': 11, 'Tomato_Healthy': 12, 'Pepper_Late_Blight': 13, 'Tomato_Late_Blight': 14, 'Pepper_Cercospora': 15, 'Tomato_Fusarium': 16, 'Pepper_Leaf_Blight': 17, 'Tomato_Leaf_Curl': 18, 'Tomato_Bacterial_Spot': 19, 'Tomato_Mosaic': 20, 'Pepper_Early_Blight': 21, 'Corn_Northern_Leaf_Blight': 22})

# Split data into training and validation
train_unique_imgs_df = train.drop_duplicates(subset=['Image_ID'], ignore_index=True)
X_train, X_val = train_test_split(train_unique_imgs_df, test_size=0.25, stratify=train_unique_imgs_df['class'], random_state=42)

X_train = train[train.Image_ID.isin(X_train.Image_ID)]
X_val = train[train.Image_ID.isin(X_val.Image_ID)]

# Check the shapes of training and validation data
print(X_train.shape, X_val.shape)

# Define directories for images and labels
TRAIN_IMAGES_DIR = Path('/content/data1/train/images')
VAL_IMAGES_DIR = Path('/content/data1/val/images')
TEST_IMAGES_DIR = Path('/content/data1/test/images')
TRAIN_LABELS_DIR = Path('/content/data1/train/labels')
VAL_LABELS_DIR = Path('/content/data1/val/labels')
TEST_LABELS_DIR = Path('/content/data1/test/labels')

# Create necessary directories
for DIR in [TRAIN_IMAGES_DIR, VAL_IMAGES_DIR, TEST_IMAGES_DIR, TRAIN_LABELS_DIR, VAL_LABELS_DIR, TEST_LABELS_DIR]:
    if DIR.exists():
        shutil.rmtree(DIR)
    DIR.mkdir(parents=True, exist_ok=True)

# Copy train, val, and test images to their respective dirs
for img in tqdm(X_train.image_path.unique()):
    shutil.copy(img, TRAIN_IMAGES_DIR / img.parts[-1])

for img in tqdm(X_val.image_path.unique()):
    shutil.copy(img, VAL_IMAGES_DIR / img.parts[-1])

for img in tqdm(test.image_path.unique()):
    shutil.copy(img, TEST_IMAGES_DIR / img.parts[-1])

# Function to convert the bounding boxes to YOLO format and save them
def save_yolo_annotation(row):
    image_path, class_id, output_dir = row['image_path'], row['class_id'], row['output_dir']

    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Could not read image from path: {image_path}")

    height, width, _ = img.shape
    label_file = Path(output_dir) / f"{Path(image_path).stem}.txt"

    ymin, xmin, ymax, xmax = row['ymin'], row['xmin'], row['ymax'], row['xmax']

    # Normalize the coordinates
    x_center = (xmin + xmax) / 2 / width
    y_center = (ymin + ymax) / 2 / height
    bbox_width = (xmax - xmin) / width
    bbox_height = (ymax - ymin) / height

    with open(label_file, 'a') as f:
        f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

# Parallelize the annotation saving process
def process_dataset(dataframe, output_dir):
    dataframe['output_dir'] = output_dir
    with multiprocessing.Pool() as pool:
        list(tqdm(pool.imap(save_yolo_annotation, dataframe.to_dict('records')), total=len(dataframe)))

# Save train and validation labels to their respective dirs
process_dataset(X_train, TRAIN_LABELS_DIR)
process_dataset(X_val, VAL_LABELS_DIR)


In [ ]:
import yaml
from pathlib import Path


# Obtenir les classes uniques de votre jeu de données
# Remplacer 'train' par votre dataframe contenant la colonne 'class'
class_names = train['class'].unique().tolist()  # Exemple si vous avez un dataframe `train` avec une colonne `class`
num_classes = len(class_names)

# Poids des classes : donnez plus de poids aux classes rares, par exemple
cls_weights = [1.0, 5.0, 10.0]  # Adaptez selon le nombre de classes et leur rareté

# Créer le fichier data.yaml
data_yaml = {
    'train': str(TRAIN_IMAGES_DIR),
    'val': str(VAL_IMAGES_DIR),
    'nc': num_classes,
    'names': class_names,
    'cls_weights': cls_weights  # Ajouter la pondération des classes
}

# Sauvegarder le fichier data.yaml
yaml_path = Path('data.yaml')
with open(yaml_path, 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

print("data.yaml créé avec succès !")


In [ ]:
import yaml

# Définitions pour le fichier data.yaml
data_yaml = {
    'cls_weights': [
        1.0, 5.0, 10.0, 1.0, 1.5, 1.0, 1.0, 1.5, 1.0, 1.0,
        1.0, 1.0, 2.0, 2.0, 1.0, 1.5, 2.0, 1.0, 2.0, 1.5,
        1.0, 1.0, 1.0
    ],
    'names': [
        'Pepper_Bacterial_Spot', 'Pepper_Fusarium', 'Corn_Cercospora_Leaf_Spot', 'Corn_Common_Rust', 'Tomato_Early_Blight',
        'Pepper_Septoria', 'Tomato_Septoria', 'Pepper_Leaf_Curl', 'Pepper_Leaf_Mosaic', 'Corn_Streak', 'Corn_Healthy',
        'Pepper_Healthy', 'Tomato_Healthy', 'Pepper_Late_Blight', 'Tomato_Late_Blight', 'Pepper_Cercospora', 'Tomato_Fusarium',
        'Pepper_Leaf_Blight', 'Tomato_Leaf_Curl', 'Tomato_Bacterial_Spot', 'Tomato_Mosaic', 'Pepper_Early_Blight',
        'Corn_Northern_Leaf_Blight'
    ],
    'nc': 23,
    'train': '/content/data1/train/images',
    'val': '/content/data1/val/images'
}

# Chemin vers le fichier data.yaml
yaml_path = '/content/data.yaml'

# Écriture dans le fichier data.yaml
with open(yaml_path, 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

print(f"Fichier {yaml_path} modifié avec succès.")


In [ ]:
from ultralytics import YOLO

# Charger le modèle pré-entraîné YOLOv8 (ici le modèle YOLOv8s comme exemple)
model = YOLO('yolov8s.pt')

# Entraînement du modèle avec le fichier data.yaml et les poids de classes
model.train(data='data.yaml', epochs=50, batch=16, imgsz=640)

# Vérifier les résultats
print("Entraînement terminé")


100%|██████████| 21.5M/21.5M [00:00<00:00, 41.9MB/s]


Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CPU (AMD EPYC 7B12)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, f

100%|██████████| 755k/755k [00:00<00:00, 112MB/s]


Overriding model.yaml nc=80 with nc=23

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

train: Scanning /content/data1/train/labels... 3676 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3676/3676 [00:20<00:00, 181.06it/s]


...

                   all       1226      10252      0.415      0.347      0.282      0.123

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      15/15         0G      1.948       1.77       1.44        147        640: 100%|██████████| 230/230 [1:01:49<00:00, 16.13s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [05:34<00:00,  8.58s/it]


                   all       1226      10252      0.414      0.348      0.291      0.128

15 epochs completed in 17.008 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CPU (AMD EPYC 7B12)
Model summary (fused): 168 layers, 11,134,485 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [04:58<00:00,  7.64s/it]


                   all       1226      10252      0.414      0.348      0.292      0.128
 Pepper_Bacterial_Spot        132        476      0.414      0.206      0.234     0.0576
       Pepper_Fusarium         52        123      0.491       0.65      0.623      0.235
Corn_Cercospora_Leaf_Spot        202       1600      0.396      0.453      0.386      0.152
      Corn_Common_Rust         79        442      0.413      0.286       0.28      0.109
   Tomato_Early_Blight         56        476      0.421      0.403      0.383      0.186
       Pepper_Septoria         40        697      0.394     0.0764      0.139     0.0505
       Tomato_Septoria        119       1588      0.385       0.29      0.242     0.0861
      Pepper_Leaf_Curl        138        374      0.322      0.297      0.222     0.0779
    Pepper_Leaf_Mosaic        147        523          1          0     0.0339    0.00884
           Corn_Streak         87        804      0.281      0.535      0.317      0.114
          Corn_Hea

In [ ]:
# Validate the model on the validation seta
model.val()

# Make predictions on test images and save them to a CSV file
image_files = os.listdir(TEST_IMAGES_DIR)
all_data = []

for image_file in tqdm(image_files):
    img_path = os.path.join(TEST_IMAGES_DIR, image_file)
    results = model(img_path)

    boxes = results[0].boxes.xyxy.tolist()
    classes = results[0].boxes.cls.tolist()
    confidences = results[0].boxes.conf.tolist()
    names = results[0].names

    incorrect_prediction = 0

    if not boxes:
        incorrect_prediction = incorrect_prediction +   1
        all_data.append({
            'Image_ID': image_file,
            'class': 'NEG',
            'confidence': 1.0,
            'ymin': 0,
            'xmin': 0,
            'ymax': 0,
            'xmax': 0
        })
    else:
        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = box
            detected_class = names[int(cls)]

            all_data.append({
                'Image_ID': image_file,
                'class': detected_class,
                'confidence': conf,
                'ymin': y1,
                'xmin': x1,
                'ymax': y2,
                'xmax': x2
            })
print(f'========== Total Incorrect predction or dummy prediction {incorrect_prediction}================')
# Convert the results to a DataFrame and save it
sub = pd.DataFrame(all_data)
sub.to_csv('/content/data1/50_epoch_ponderation_submission.csv', index=False)


Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CPU (AMD EPYC 7B12)
Model summary (fused): 168 layers, 11,134,485 parameters, 0 gradients, 28.5 GFLOPs


val: Scanning /content/data1/val/labels.cache... 1226 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1226/1226 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 77/77 [05:00<00:00,  3.91s/it]


                   all       1226      10252      0.415      0.348      0.292      0.128
 Pepper_Bacterial_Spot        132        476      0.418      0.208      0.234     0.0579
       Pepper_Fusarium         52        123      0.491       0.65      0.623      0.235
Corn_Cercospora_Leaf_Spot        202       1600      0.396      0.453      0.386      0.152
      Corn_Common_Rust         79        442      0.413      0.286       0.28      0.109
   Tomato_Early_Blight         56        476      0.421      0.403      0.383      0.186
       Pepper_Septoria         40        697      0.394     0.0764      0.139     0.0504
       Tomato_Septoria        119       1588      0.386      0.292      0.242     0.0861
      Pepper_Leaf_Curl        138        374      0.321      0.294      0.224      0.079
    Pepper_Leaf_Mosaic        147        523          1          0     0.0341    0.00886
           Corn_Streak         87        804      0.281      0.535      0.317      0.114
          Corn_Hea

  0%|          | 0/2101 [00:00<?, ?it/s]

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Speed: 2.3ms preprocess, 230.0ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /content/data1/test/images/id_gumdcy.jpg: 480x640 8 Corn_Streaks, 220.0ms
Speed: 2.6ms preprocess, 220.0ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/data1/test/images/id_ere3cz.jpg: 448x640 3 Tomato_Septorias, 206.0ms
Speed: 2.1ms preprocess, 206.0ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /content/data1/test/images/id_opk7co.jpg: 480x640 1 Corn_Streak, 214.2ms
Speed: 2.7ms preprocess, 214.2ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/data1/test/images/id_4gtki1.jpg: 480x640 4 Tomato_Early_Blights, 1 Tomato_Late_Blight, 221.6ms
Speed: 2.6ms preprocess, 221.6ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/data1/test/images/id_3r9xg4.jpg: 480x64

In [ ]:
# prompt: download /content/data1/benchmark_submission.csv

from google.colab import files
files.download('/content/data1/50_epoch_ponderation_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>